# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import time

# generic ros libraries
import rclpy
from rclpy.logging import get_logger

# moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (
    MoveItPy,
    MultiPipelinePlanRequestParameters,
)

from ament_index_python.packages import get_package_share_directory
from moveit_configs_utils import MoveItConfigsBuilder

def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    """Helper function to plan and execute a motion."""
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(robot_name="lite6", package_name="moveit_resources_lite6_moveit_config")
        .robot_description_semantic(file_path="config/lite6.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path=get_package_share_directory("moveit_resources_lite6_description")
                          + "/urdf/lite6.urdf")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_motion_planning_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

## Setup

In [2]:
rclpy.init()
logger = get_logger("moveit_py.pose_goal")
    
# instantiate MoveItPy instance and get planning component
lite6 = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6_arm = lite6.get_planning_component("lite6")
logger.info("MoveItPy instance created")

[INFO] [1698419602.810067021] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1698419602.810689326] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1698419602.810702646] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1698419602.826824884] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1698419602.828973049] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.00199817 seconds
[INFO] [1698419602.829034109] [moveit_robot_model.robot_model]: Loading robot model 'UF_ROBOT'...
[INFO] [1698419602.829049490] [moveit_robot_model.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1698419602.863930963] [moveit_kinematics_base.kinematics_base]: Joint weights for group 'lite6': 1 1 1 1 1 1
[INFO] [1698419603.228433133] [moveit_ros.planning_scene_monitor.planning_scene_monitor]: Publishing maintained planning scene on 'monitored_planning_scene'
[INFO] [1698419603.228557433] [moveit.ros_planning_interface.moveit_cpp]: Listen

True

## Motion Planning Example

In [ ]:
###########################################################################
# Plan 1 - set states with predefined string
###########################################################################

# set plan start state to current state
lite6_arm.set_start_state_to_current_state()

# set pose goal using predefined state
lite6_arm.set_goal_state(configuration_name="home")

# plan to goal
plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

In [4]:
 ###########################################################################
# Plan 2 - set goal state with RobotState object
###########################################################################

# instantiate a RobotState instance using the current robot model
robot_model = lite6.get_robot_model()
robot_state = RobotState(robot_model)

# randomize the robot state
robot_state.set_to_random_positions()

# set plan start state to current state
lite6_arm.set_start_state_to_current_state()

# set goal state to the initialized robot state
logger.info("Set goal state to the initialized robot state")
lite6_arm.set_goal_state(robot_state=robot_state)

# plan to goal
plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

[INFO] [1698419607.589602005] [moveit_py.pose_goal]: Set goal state to the initialized robot state
[INFO] [1698419607.592104313] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1698419607.592152113] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1698419607.596549626] [moveit.pilz_industrial_motion_planner.trajectory_generator_ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1698419607.596600056] [moveit.pilz_industrial_motion_planner.trajectory_generator]: Generating PTP trajectory...
[INFO] [1698419607.673917607] [moveit_py.pose_goal]: Executing plan
[INFO] [1698419607.674062458] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419607.674098178] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419607.674386590] [moveit.plugins.moveit_simple_controller_manager]: Retur

In [5]:
###########################################################################
# Plan 3 - set goal state with PoseStamped message
###########################################################################

# set plan start state to current state
lite6_arm.set_start_state_to_current_state()

# set pose goal with PoseStamped message
from geometry_msgs.msg import PoseStamped

pose_goal = PoseStamped()
pose_goal.header.frame_id = "link_base"
pose_goal.pose.orientation.w = 1.0
pose_goal.pose.position.x = 0.28
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.5
lite6_arm.set_goal_state(pose_stamped_msg=pose_goal, pose_link="link6")

# plan to goal
plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

[INFO] [1698419620.287868275] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1698419620.287921056] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1698419620.291609102] [moveit.pilz_industrial_motion_planner.trajectory_generator_ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1698419620.291654042] [moveit.pilz_industrial_motion_planner.trajectory_generator]: Generating PTP trajectory...
[INFO] [1698419620.388274505] [moveit_py.pose_goal]: Executing plan
[INFO] [1698419620.388376615] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419620.388405536] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419620.388733428] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419620.388746138] [moveit.plugins.moveit_simple_controller_ma

In [6]:
###########################################################################
# Plan 4 - set goal state with constraints
###########################################################################

# set plan start state to current state
lite6_arm.set_start_state_to_current_state()

# set constraints message
from moveit.core.kinematic_constraints import construct_joint_constraint

joint_values = {
    "joint1": 1.0,
    "joint2": 0.0,
    "joint3": 0.0,
    "joint4": 1.0,
    "joint5": 0.0,
    "joint6": 0.0,
}
robot_state.joint_positions = joint_values
joint_constraint = construct_joint_constraint(
    robot_state=robot_state,
    joint_model_group=lite6.get_robot_model().get_joint_model_group("lite6"),
)
lite6_arm.set_goal_state(motion_plan_constraints=[joint_constraint])

# plan to goal
plan_and_execute(lite6, lite6_arm, logger, sleep_time=3.0)

[INFO] [1698419635.900289217] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1698419635.900338068] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1698419635.903607181] [moveit.pilz_industrial_motion_planner.trajectory_generator_ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1698419635.903646651] [moveit.pilz_industrial_motion_planner.trajectory_generator]: Generating PTP trajectory...
[INFO] [1698419635.943974197] [moveit_py.pose_goal]: Executing plan
[INFO] [1698419635.944074538] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419635.944103958] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419635.944254019] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1698419635.944266369] [moveit.plugins.moveit_simple_controller_ma

In [8]:
###########################################################################
# Plan 5 - Planning with Multiple Pipelines simultaneously
###########################################################################

# set plan start state to current state
lite6_arm.set_start_state_to_current_state()

# set pose goal with PoseStamped message
lite6_arm.set_goal_state(configuration_name="home")

# initialise multi-pipeline plan request parameters
multi_pipeline_plan_request_params = MultiPipelinePlanRequestParameters(
    lite6, ["ompl_rrtc", "pilz_lin", "chomp_planner"]
)

# plan to goal
plan_and_execute(
    lite6,
    lite6_arm,
    logger,
    multi_plan_parameters=multi_pipeline_plan_request_params,
    sleep_time=3.0,
)


[WARN] [1698419651.948816877] [moveit_py]: Parameter 'ompl_rrtc.plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1698419651.948852548] [moveit_py]: Parameter 'ompl_rrtc.plan_request_params.planning_pipeline' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1698419651.948858768] [moveit_py]: Parameter 'ompl_rrtc.plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1698419651.948863968] [moveit_py]: Parameter 'ompl_rrtc.plan_request_params.planning_attempts' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1698419651.948868288] [moveit_py]: Parameter 'ompl_rrtc.plan_request_params.max_velocity_scaling_factor' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [169